# 🚗 Indian License Plate Detector - YOLOv8

**Author:** Himanshu Sharma  
**Project:** Smart Traffic Detection System

---

## ⚡ Before Starting
**Enable GPU:** Runtime → Change runtime type → **T4 GPU**

In [ ]:
#@title ✅ Step 1: Install Dependencies
!pip install ultralytics roboflow -q

import torch
print(f"✅ GPU: {torch.cuda.is_available()} - {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NO GPU!'}")

In [ ]:
#@title 📥 Step 2: Download Dataset (Roboflow)
from roboflow import Roboflow

# Your API Key (pre-filled)
rf = Roboflow(api_key="qSdg6sVuzoFt3DoaJ1Zx")
project = rf.workspace("yolov8-i4lu9").project("indian-number-plates-9oobq-nwbrn")
dataset = project.version(1).download("yolov8")

print(f"\n✅ Downloaded to: {dataset.location}")

In [ ]:
#@title 🔍 Step 3: Verify Dataset
import os

print("📂 Contents:")
!ls -la {dataset.location}/

print("\n📝 data.yaml:")
!cat {dataset.location}/data.yaml

In [ ]:
#@title 🎯 Step 4: Train YOLOv8 (~30-45 min)
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    device=0,
    project='runs/plate',
    name='detector',
    exist_ok=True,
    flipud=0.0,
    fliplr=0.5,
)

print("\n🎉 Training Complete!")

In [ ]:
#@title 📊 Step 5: View Results
from IPython.display import Image, display
from ultralytics import YOLO

display(Image('runs/plate/detector/results.png', width=900))

model = YOLO('runs/plate/detector/weights/best.pt')
metrics = model.val()

print(f"\n🎯 mAP50: {metrics.box.map50:.1%}")
print(f"🎯 Precision: {metrics.box.mp:.1%}")
print(f"🎯 Recall: {metrics.box.mr:.1%}")

In [ ]:
#@title 🌐 Step 6: Export to TensorFlow.js
from ultralytics import YOLO

model = YOLO('runs/plate/detector/weights/best.pt')
model.export(format='tfjs')
print("✅ Exported!")

In [ ]:
#@title 📥 Step 7: Download Model
import shutil
from google.colab import files

shutil.make_archive('plate_detector_tfjs', 'zip', 'runs/plate/detector/weights/best_web_model')
shutil.copy('runs/plate/detector/weights/best.pt', 'plate_detector.pt')

files.download('plate_detector_tfjs.zip')
files.download('plate_detector.pt')
print("\n✅ Copy to: frontend/public/models/plate_detector/")